In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objs as go
pio.renderers.default='notebook'
#sns.set()
import random
import numpy as np
import pandas as pd
from torch.nn.functional import conv1d
import plotly.express as px
from tqdm import tqdm
import time

import multichange as mc
from importlib import reload
reload(mc);

In [ ]:
n = 200
p = 100
trials = 500
r = 20
pathPNG = "/home/emmanuel/these_epilliat/changepoint-detection/revision3"
pathCsv = "./data"
titleloss = r"$\text{Estimation of the risk }\mathbb{E}\left[\mathbf{SAND}(\{\hat \tau_k \}, \{\tau_k\})\right]$"
titlepdiff = r"$\text{Estimation of the probability }\mathbb{P}(K \neq \hat K)$"
ytitle="risk"

## Segment setting

In [ ]:
for s in [1, 20, p]:
    df = pd.read_csv(pathCsv + f"/n{n}_p{p}_s{s}_r{r}_trials{trials}.csv")
    df = df.rename({"Unnamed: 0": "Norm"}, axis=1)
    df = df.set_index("Norm")
    
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=1, cols=2)
    fig = go.Figure(
        [
            go.Scatter(
                name="bottomup",
                x=df.index,
                y=df["pdiff_bottomup"],
                mode="lines",
                line=dict(color="red"),
            ),
            go.Scatter(
                name="inspect",
                x=df.index,
                y=df["pdiff_inspect"],
                mode="lines",
                line=dict(color="blue", dash='dot'),
            ),
            go.Scatter(
                name="LBbottomup",
                x=df.index,
                y=(df["pdiff_bottomup"] - df["confidence_bound_bottomup"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBbottomup",
                x=df.index,
                y=(df["pdiff_bottomup"] + df["confidence_bound_bottomup"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(150,0,0,0.3)",
                fill="tonexty",
                showlegend=False,
            ),
            go.Scatter(
                name="LBinspect",
                x=df.index,
                y=(df["pdiff_inspect"] - df["confidence_bound_inspect"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBinspect",
                x=df.index,
                y=(df["pdiff_inspect"] + df["confidence_bound_inspect"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(0,0,150,0.3)",
                fill="tonexty",
                showlegend=False,
            ),
        ]
    )
    fig.update_layout(
        title=titlepdiff,
        xaxis_title="scaling factor",
        yaxis_title=ytitle,
    )
    fig.write_image(pathPNG + f"/pdiff_n{n}_p{p}_s{s}_r{r}_trials{trials}.png")


In [ ]:
for s in [1, 20, p]:
    df = pd.read_csv(pathCsv + f"/n{n}_p{p}_s{s}_r{r}_trials{trials}.csv")
    df = df.rename({"Unnamed: 0": "Norm"}, axis=1)
    df = df.set_index("Norm")
    
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=1, cols=2)
    fig = go.Figure(
        [
            go.Scatter(
                name="bottomup",
                x=df.index,
                y=df["loss_bottomup"],
                mode="lines",
                line=dict(color="red"),
            ),
            go.Scatter(
                name="inspect",
                x=df.index,
                y=df["loss_inspect"],
                mode="lines",
                line=dict(color="blue", dash='dot'),
            ),
            go.Scatter(
                name="LBbottomup",
                x=df.index,
                y=(df["loss_bottomup"] - 1.96*1/np.sqrt(500)).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBbottomup",
                x=df.index,
                y=(df["loss_bottomup"] + 1.96*1/np.sqrt(500)).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(150,0,0,0.2)",
                fill="tonexty",
                showlegend=False,
            ),
            go.Scatter(
                name="LBinspect",
                x=df.index,
                y=(df["loss_inspect"] - 1.96*1/np.sqrt(500)).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBinspect",
                x=df.index,
                y=(df["loss_inspect"] + 1.96*1/np.sqrt(500)).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(0,0,150,0.2)",
                fill="tonexty",
                showlegend=False,
            ),
        ]
    )
    fig.update_layout(
        title=titleloss,
        xaxis_title="scaling factor",
        yaxis_title=ytitle,
    )
    fig.write_image(pathPNG + f"/loss_n{n}_p{p}_s{s}_r{r}_trials{trials}.png")


## Multi change-points setting

In [ ]:
df = pd.read_csv(pathCsv + f"/K{10}_n{n}_p{p}_trials{trials}.csv")
df = df.rename({"Unnamed: 0": "Norm"}, axis=1)
df = df.set_index("Norm")

import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2)
fig = go.Figure(
    [
        go.Scatter(
            name="bottomup",
            x=df.index,
            y=df["pdiff_bottomup"],
            mode="lines",
            line=dict(color="red"),
        ),
        go.Scatter(
            name="inspect",
            x=df.index,
            y=df["pdiff_inspect"],
            mode="lines",
            line=dict(color="blue", dash='dot'),
        ),
        go.Scatter(
            name="LBbottomup",
            x=df.index,
            y=(df["pdiff_bottomup"] - df["confidence_bound_bottomup"]).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            showlegend=False,
        ),
        go.Scatter(
            name="UBbottomup",
            x=df.index,
            y=(df["pdiff_bottomup"] + df["confidence_bound_bottomup"]).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba(150,0,0,0.2)",
            fill="tonexty",
            showlegend=False,
        ),
        go.Scatter(
            name="LBinspect",
            x=df.index,
            y=(df["pdiff_inspect"] - df["confidence_bound_bottomup"]).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            showlegend=False,
        ),
        go.Scatter(
            name="UBinspect",
            x=df.index,
            y=(df["pdiff_inspect"] + df["confidence_bound_bottomup"]).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba(0,0,150,0.2)",
            fill="tonexty",
            showlegend=False,
        ),
    ]
)
fig.update_layout(
    title=titlepdiff,
    xaxis_title="scaling factor",
    yaxis_title=ytitle,
)
fig.write_image(pathPNG + f"/pdiff_K{10}_n{n}_p{p}_trials{trials}.png")
fig.show()

In [ ]:
df = pd.read_csv(pathCsv + f"/K{10}_n{n}_p{p}_trials{trials}.csv")
df = df.rename({"Unnamed: 0": "Norm"}, axis=1)
df = df.set_index("Norm")

import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2)
fig = go.Figure(
    [
        go.Scatter(
            name="bottomup",
            x=df.index,
            y=df["loss_bottomup"],
            mode="lines",
            line=dict(color="red"),
        ),
        go.Scatter(
            name="inspect",
            x=df.index,
            y=df["loss_inspect"],
            mode="lines",
            line=dict(color="blue", dash='dot'),
        ),
        go.Scatter(
            name="LBbottomup",
            x=df.index,
            y=(df["loss_bottomup"] - 1.96*1/np.sqrt(500)).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            showlegend=False,
        ),
        go.Scatter(
            name="UBbottomup",
            x=df.index,
            y=(df["loss_bottomup"] + 1.96*1/np.sqrt(500)).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba(150,0,0,0.2)",
            fill="tonexty",
            showlegend=False,
        ),
        go.Scatter(
            name="LBinspect",
            x=df.index,
            y=(df["loss_inspect"] - 1.96*1/np.sqrt(500)).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            showlegend=False,
        ),
        go.Scatter(
            name="UBinspect",
            x=df.index,
            y=(df["loss_inspect"] + 1.96*1/np.sqrt(500)).clip(0, 1),
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba(0,0,150,0.2)",
            fill="tonexty",
            showlegend=False,
        ),
    ]
)
fig.update_layout(
    title=titleloss,
    xaxis_title="scaling factor",
    yaxis_title=ytitle,
)
fig.write_image(pathPNG + f"/loss_K{10}_n{n}_p{p}_trials{trials}.png")
fig.show()


## Time dependent setting

In [ ]:
rho = 0.05
trials = 500
for s in [20]:
    df = pd.read_csv(pathCsv + f"/time_dependent_s{s}_rho{int(100*rho)}_trials{trials}.csv")
    df = df.rename({"Unnamed: 0": "Norm"}, axis=1)
    df = df.set_index("Norm")
    
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=1, cols=2)
    fig = go.Figure(
        [
            go.Scatter(
                name="bottomup",
                x=df.index,
                y=df["pdiff_bottomup"],
                mode="lines",
                line=dict(color="red"),
            ),
            go.Scatter(
                name="inspect",
                x=df.index,
                y=df["pdiff_inspect"],
                mode="lines",
                line=dict(color="blue", dash='dot'),
            ),
            go.Scatter(
                name="LBbottomup",
                x=df.index,
                y=(df["pdiff_bottomup"] - df["confidence_bound_bottomup"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBbottomup",
                x=df.index,
                y=(df["pdiff_bottomup"] + df["confidence_bound_bottomup"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(150,0,0,0.3)",
                fill="tonexty",
                showlegend=False,
            ),
            go.Scatter(
                name="LBinspect",
                x=df.index,
                y=(df["pdiff_inspect"] - df["confidence_bound_inspect"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBinspect",
                x=df.index,
                y=(df["pdiff_inspect"] + df["confidence_bound_inspect"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(0,0,150,0.3)",
                fill="tonexty",
                showlegend=False,
            ),
        ]
    )
    fig.update_layout(
        title=titlepdiff,
        xaxis_title="scaling factor",
        yaxis_title=ytitle,
    )
    fig.write_image(pathPNG + f"/pdiff_time_dependent_s{s}_rho{int(100*rho)}_trials{trials}.png")
fig.show()


In [ ]:
rho = 0.05
trials = 500
for s in [20]:
    df = pd.read_csv(pathCsv + f"/time_dependent_s{s}_rho{int(100*rho)}_trials{trials}.csv")
    df = df.rename({"Unnamed: 0": "Norm"}, axis=1)
    df = df.set_index("Norm")
    
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=1, cols=2)
    fig = go.Figure(
        [
            go.Scatter(
                name="bottomup",
                x=df.index,
                y=df["loss_bottomup"],
                mode="lines",
                line=dict(color="red"),
            ),
            go.Scatter(
                name="inspect",
                x=df.index,
                y=df["loss_inspect"],
                mode="lines",
                line=dict(color="blue", dash='dot'),
            ),
            go.Scatter(
                name="LBbottomup",
                x=df.index,
                y=(df["loss_bottomup"] - df["confidence_bound_bottomup"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBbottomup",
                x=df.index,
                y=(df["loss_bottomup"] + df["confidence_bound_bottomup"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(150,0,0,0.3)",
                fill="tonexty",
                showlegend=False,
            ),
            go.Scatter(
                name="LBinspect",
                x=df.index,
                y=(df["loss_inspect"] - df["confidence_bound_inspect"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                showlegend=False,
            ),
            go.Scatter(
                name="UBinspect",
                x=df.index,
                y=(df["loss_inspect"] + df["confidence_bound_inspect"]).clip(0, 1),
                marker=dict(color="#444"),
                line=dict(width=0),
                mode="lines",
                fillcolor="rgba(0,0,150,0.3)",
                fill="tonexty",
                showlegend=False,
            ),
        ]
    )
    fig.update_layout(
        title=titleloss,
        xaxis_title="scaling factor",
        yaxis_title=ytitle,
    )
    fig.write_image(pathPNG + f"/loss_time_dependent_s{s}_rho{int(100*rho)}_trials{trials}.png")
fig.show()